<a href="https://colab.research.google.com/github/i-moes/TM10007_PROJECT/blob/master/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM10007 Assignment template

In [1]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/i-moes/TM10007_PROJECT.git

# Install packages
!pip install sklearn numpy matplotlib

In [0]:
# # General packages
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets as ds
from sklearn import metrics
from sklearn.model_selection import train_test_split
 
# Metrics
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import r2_score

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [19]:
# Data loading functions. Uncomment the one you want to use
#from adni.load_data import load_data
from brats.load_data import load_data
#from hn.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

print(data.head())

The number of samples: 167
The number of columns: 725
              VOLUME_ET  VOLUME_NET  VOLUME_ED  ...  TGM_Cog_Z_6  TGM_T_6  label
ID                                              ...                             
TCGA-02-0006       1662         384      36268  ...          NaN      NaN    GBM
TCGA-02-0009       4362        4349      15723  ...          NaN      NaN    GBM
TCGA-02-0011      33404       48612      45798  ...          NaN      NaN    GBM
TCGA-02-0027      12114        7587      34086  ...          NaN      NaN    GBM
TCGA-02-0033      34538        7137      65653  ...          NaN      NaN    GBM

[5 rows x 725 columns]


**1 Splitting Training and Test set**

First the data needs to be splitted into a train and test set. This is done using train_test_split from sklearn.model_selection. The test data contains 45% of the data. 

In [0]:


# splitting data in train and test set 
x_train, x_test = train_test_split(data, test_size = 0.45)

print(x_train.head())
print(x_test.head())



In [0]:
# extracting labels from both the train and test set
y_train = x_train['label']
y_test = x_test['label']
# removing label column from dataframes
del x_train['label']
del x_test['label']


**2 Preprocessing**

Next, ...

In [0]:
# preprocessing data

**3 Training classifiers**

Different classifiers were trained. 

In [0]:
# training

**4 Performance evaluation**

The classifiers were then tested on the test set. The following evaluation measures were used:


*   Accuracy
*   F1-Score
*  AUC



In [0]:
# Metrics 